In [1]:
# libraries for nlp
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# libraries required for Tensorflow processing
import tensorflow as tf
import tflearn
import numpy as np
import json
import random

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADITYA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
W0817 13:57:42.146343  8036 deprecation_wrapper.py:119] From c:\users\aditya\appdata\local\programs\python\python37\lib\site-packages\tflearn\helpers\summarizer.py:9: The name tf.summary.merge is deprecated. Please use tf.compat.v1.summary.merge instead.

W0817 13:57:42.161944  8036 deprecation_wrapper.py:119] From c:\users\aditya\appdata\local\programs\python\python37\lib\site-packages\tflearn\helpers\trainer.py:25: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

W0817 13:57:42.193113  8036 deprecation_wrapper.py:119] From c:\users\aditya\appdata\local\programs\python\python37\lib\site-packages\tflearn\collections.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0817 13:57:42.224346  8036 deprecation_wrapper.py:119] From c:\users\aditya\appdata\

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
print(tf.__version__)

1.14.0


In [8]:
with open('intents.json') as json_data:
    intents = json.load(json_data)
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Hello', 'Is anyone there?', 'Good day'],
   'responses': ['Hi there, how can I help?',
    'Hello, thanks for visiting',
    'Good to see you again'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Goodbye', 'Bye', 'See you later'],
   'responses': ['Have a nice day',
    'See you later, thanks for visiting',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thank you', 'Thanks', "That's helpful"],
   'responses': ['No problem', 'My pleasure', 'Happy to help!', 'Any time!']},
  {'tag': 'hours',
   'patterns': ['When are you open?',
    'What hours are you open?',
    'What are your hours?'],
   'responses': ['Our hours are 9am-5pm every day',
    "We're open every day 9am-5pm"]},
  {'tag': 'location',
   'patterns': ['Where are you located?',
    'What is your address?',
    'What is the location of your clinic?'],
   'responses': ['Our address is Connaught Place New Delhi',
    

In [13]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence present in 'patterns' of 'intents' 
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # create tokens for each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to the words list
        words.extend(w)
        # add to the documents list
        documents.append((w, intent['tag']))
        # add to the classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [16]:
# perform stemming on every word and remove duplicate words
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicate classes using set
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

28 documents
9 classes ['goodbye', 'greeting', 'hours', 'location', 'opentoday', 'payments', 'tests', 'thanks', 'today']
48 unique stemmed words ["'s", 'acceiv', 'address', 'anyon', 'ar', 'at', 'avail', 'bye', 'card', 'cash', 'clin', 'credit', 'day', 'do', 'don', 'good', 'goodby', 'hello', 'help', 'hi', 'hour', 'how', 'is', 'lat', 'loc', 'mastercard', 'mobl', 'of', 'on', 'op', 'pay', 'see', 'serv', 'tak', 'test', 'thank', 'that', 'the', 'ther', 'through', 'today', 'wallet', 'what', 'when', 'wher', 'which', 'yo', 'you']


In [20]:
# create a list to store the training data
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create the bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output '1' for current tag and '0' for other tags 
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle the features and turn it into a numpy array
random.shuffle(training)
training = np.array(training)

# create train and test data
train_x = list(training[:,0])
test_x = list(training[:,1])

In [21]:
# reset underlying graph data
tf.reset_default_graph()

In [22]:
# build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(test_x[0]), activation='softmax')
net = tflearn.regression(net)

W0817 16:08:27.622880  8036 deprecation_wrapper.py:119] From c:\users\aditya\appdata\local\programs\python\python37\lib\site-packages\tflearn\layers\core.py:81: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0817 16:08:28.591504  8036 deprecation.py:506] From c:\users\aditya\appdata\local\programs\python\python37\lib\site-packages\tflearn\initializations.py:174: calling TruncatedNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0817 16:08:30.292330  8036 deprecation_wrapper.py:119] From c:\users\aditya\appdata\local\programs\python\python37\lib\site-packages\tflearn\optimizers.py:238: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

W0817 16:08:30.462964  8036 deprecation.py:506] From c:\users\aditya\appdata\lo

In [23]:
# define model, setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# train the model with gradient descent algorithm
model.fit(train_x, test_x, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3849  | total loss: 0.70686 | time: 0.009s
| Adam | epoch: 963 | loss: 0.70686 - acc: 0.9043 -- iter: 08/28


In [32]:
# save all the data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'test_x':test_x}, open( "training_data", "wb" ) )

In [33]:
# restore all the data structures
data = pickle.load( open("training_data", "rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
test_x = data['test_x']

In [34]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [35]:
# load the saved model
model.load('./model.tflearn')

In [38]:
def clean_up_sentence(sentence):
    # tokenize the sentences
    token_words = nltk.word_tokenize(sentence)
    # stem each word present as tokens
    token_words = [stemmer.stem(word.lower()) for word in token_words]
    return token_words

In [44]:
# generate 'bag of words' array containg 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the sentences
    token_words = clean_up_sentence(sentence)
    # generate bag of words
    bag = [0]*len(words)  
    for s in token_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("Found in bag: %s" % w)

    return(np.array(bag))

In [48]:
s = bow("what is the location of your clinic?", words)
print(s)
print(classes)

[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 1 0 0 0 1 0]
['goodbye', 'greeting', 'hours', 'location', 'opentoday', 'payments', 'tests', 'thanks', 'today']
